In [51]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.metrics import mean_squared_error,r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error,mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor 
import warnings
import logger

In [13]:
df=pd.read_csv("data/StudentsPerformance.csv")

In [14]:
df['total score']=df['math score']+df['reading score']+df['writing score']
df['average']=df['total score']/3

In [15]:
df

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score,total score,average
0,female,group B,bachelor's degree,standard,none,72,72,74,218,72.666667
1,female,group C,some college,standard,completed,69,90,88,247,82.333333
2,female,group B,master's degree,standard,none,90,95,93,278,92.666667
3,male,group A,associate's degree,free/reduced,none,47,57,44,148,49.333333
4,male,group C,some college,standard,none,76,78,75,229,76.333333
...,...,...,...,...,...,...,...,...,...,...
995,female,group E,master's degree,standard,completed,88,99,95,282,94.000000
996,male,group C,high school,free/reduced,none,62,55,55,172,57.333333
997,female,group C,high school,free/reduced,completed,59,71,65,195,65.000000
998,female,group D,some college,standard,completed,68,78,77,223,74.333333


I will be predicting total score by using all the other as independent features.

In [16]:
x=df.drop(columns=['total score'],axis=1)
y=df['total score']

In [17]:
print('Categories present in gender variable:',end=" ")
print(df['gender'].unique())

print('Categories present in race and ethnicity variable:',end=" ")
print(df['race/ethnicity'].unique())

print('Categories present in parental level of education variable:',end=" ")
print(df['parental level of education'].unique())

print('Categories present in test prep course variable:',end=" ")
print(df['test preparation course'].unique())

Categories present in gender variable: ['female' 'male']
Categories present in race and ethnicity variable: ['group B' 'group C' 'group A' 'group D' 'group E']
Categories present in parental level of education variable: ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
Categories present in test prep course variable: ['none' 'completed']


Since the number of categories are less, we can use one hot encoder, and we do scaling for numerical features


In [24]:
num_features=x.select_dtypes(exclude="object").columns
cat_features=x.select_dtypes(include="object").columns

from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer

numeric_transformer=StandardScaler()
cat_transformer=OneHotEncoder()

preprocessor=ColumnTransformer([
        ("OneHotEncoder",cat_transformer,cat_features),
        ("StandardScaler",numeric_transformer,num_features),
]
)

In [25]:
x=preprocessor.fit_transform(x)

In [26]:
x

array([[ 1.        ,  0.        ,  0.        , ...,  0.19399858,
         0.39149181,  0.34357423],
       [ 1.        ,  0.        ,  0.        , ...,  1.42747598,
         1.31326868,  1.0219275 ],
       [ 1.        ,  0.        ,  0.        , ...,  1.77010859,
         1.64247471,  1.74706375],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.12547206,
        -0.20107904, -0.19443008],
       [ 1.        ,  0.        ,  0.        , ...,  0.60515772,
         0.58901542,  0.46053169],
       [ 1.        ,  0.        ,  0.        , ...,  1.15336989,
         1.18158627,  1.06871048]])

In [30]:
from sklearn.model_selection import train_test_split
xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.2,random_state=42)
xtrain.shape,xtest.shape

((800, 21), (200, 21))

In [40]:
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [49]:
models={
    "LinearRegression":LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "KNN":KNeighborsRegressor(),
    "DT":DecisionTreeRegressor(),
    "RF":RandomForestRegressor(),
    "XGB":XGBRegressor(),
    "Adaboost":AdaBoostRegressor(),
    "Catboost":CatBoostRegressor()
}

model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(xtrain,ytrain)

    y_train_pred=model.predict(xtrain)
    y_test_pred=model.predict(xtest)

    model_train_mae, model_train_rmse, model_train_r2=evaluate_model(ytrain,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(ytest,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model performance for training set")
    print("RMSE:{:.4f}".format(model_train_rmse))
    print("MAE:{:.4f}".format(model_train_mae))
    print("R2 Score:{:.4f}".format(model_train_r2))

    print('------------------------------')

    print("Model Performace for Test Set")
    print("RMSE:{:.4f}".format(model_test_rmse))
    print("MAE:{:.4f}".format(model_test_mae))
    print("R2 Score:{:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*35)
    print('\n')

LinearRegression
Model performance for training set
RMSE:0.0000
MAE:0.0000
R2 Score:1.0000
------------------------------
Model Performace for Test Set
RMSE:0.0000
MAE:0.0000
R2 Score:1.0000


Lasso
Model performance for training set
RMSE:1.0090
MAE:0.8075
R2 Score:0.9994
------------------------------
Model Performace for Test Set
RMSE:1.0557
MAE:0.8312
R2 Score:0.9994


Ridge
Model performance for training set
RMSE:0.0176
MAE:0.0143
R2 Score:1.0000
------------------------------
Model Performace for Test Set
RMSE:0.0193
MAE:0.0147
R2 Score:1.0000


KNN
Model performance for training set
RMSE:5.9460
MAE:4.6730
R2 Score:0.9803
------------------------------
Model Performace for Test Set
RMSE:7.9983
MAE:5.7690
R2 Score:0.9668


DT
Model performance for training set
RMSE:0.0000
MAE:0.0000
R2 Score:1.0000
------------------------------
Model Performace for Test Set
RMSE:2.1564
MAE:0.3200
R2 Score:0.9976


RF
Model performance for training set
RMSE:0.4144
MAE:0.0803
R2 Score:0.9999
-------

In [50]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=["Model Name","R2_Score"]).sort_values(by=["R2_Score"],ascending=False)

,Model Name,R2_Score
0,LinearRegression,1.000000
2,Ridge,1.000000
1,Lasso,0.999422
4,DT,0.997590
6,XGB,0.997429
5,RF,0.996103
8,Catboost,0.994960
7,Adaboost,0.989709
3,KNN,0.966842


Since I am getting a 100% accuracy for train and test data, I will be using Linear Regression and Ridge as well 